<a href="https://colab.research.google.com/github/gkrry2723/algorithm_s/blob/master/Algo_Compression(Huff%26RunL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Huffman Coding**
  - 무손실 데이터 압축
  - prefix(하나의 이진트리로 나타 낼 수 있음 - 모두 leaf) code
  - 가장 짧게 압축하는 방법
  - 구성 문자들을 빈도를 기준으로 이진트리를 만들어 codeword 만듦
  - 이진트리의 모양은 다양하게 만들어질 수 있다. 단 압축 정도는 모든 트리가 정확히 동일하다.
  - 

**Run Length Encoding**
  - run: 동일한 문자가 하나 혹은 그 이상 연속하여 나오는 것
  - run을 run을 구성하는 문자, run의 길이로 나타낸다.
  

**구현할 것: Huffman Method with Run-Length Encoding**
  - huffman을 각각의 문자에 하는게 아니라 런들에게 적용
  - 각각의 run들을 super-symbol로 보고 이들에 대해 huffman coding을 적용
  - 단계
    - Run과 frequency 찾기

In [ ]:
import os

class Run:
    def __init__(self,symbol,runLen,freq):
        self.symbol = symbol
        self.runLen = runLen
        self.freq = freq
        self.left = None
        self.right = None
        self.codeword = " "
        self.codewordLen = 0

  
    def equals(self,run):
        if self.symbol == run.symbol and self.runLen == run.runLen:
            return True
        return False


#데이터 파일 읽기 -> 텍스트, 바이너리 다 ... 
# 2번 읽어야함.
# huffman code 만들고, 만든걸 배경으로 압축 이렇게 2번.


############################################# 1. run -> super symbol 모으기 ##################################
fileName = "txt.txt"
f = open(fileName, 'r')
ch = f.read(1)
start = 0
dic = {}

# 한 byte를 읽어온다.
while ch != "": 
    #지금이 시작이면
    if start == 0:
        startS = ch
        runL = 0
        start = 1
    #계속이어짐    
    if ch == startS:
        runL += 1
    #run 끝남
    if ch != startS:
        start = 0
        key = dic.get(startS*runL)
        if key: #있으면
            key[0] += 1 #frequency 
        else:
            dic[startS*runL] = [1,runL]
        continue
    ch = f.read(1)
key = dic.get(startS*runL)
if key: #있으면
    key[0] += 1 #frequency 
else:
    dic[startS*runL] = [1,runL]   
#################################### 2. 모은 super symbol 실제 run 오브젝트 만들기 ##################################

runs = []
for key, item in dic.items():
    runs.append(Run(key[0],item[1],item[0]))

############################################# 3. Huffman Tree 만들기 ###############################################
runs.sort(key = lambda x: x.freq)
# for i in range(len(runs)):
#     print(runs[i].symbol,runs[i].freq,runs[i].runLen)

def makeHeap(runs):
    Heap = [run for run in runs]
    Heap = [0] + Heap
    for i in reversed(range(len(Heap),0)):
        minHeapify(Heap,i)
    return Heap[1:]

def extractMin(Heap):
    min = Heap[0]
    Heap[0] = Heap[-1]
    Heap.pop()
    Heap = [0]+Heap
    Heap = minHeapify(Heap,1)

    return min , Heap[1:]

def printHeap(Heap):
    print("힙:")
    for i in range(1,len(Heap)):
        print(Heap[i].symbol,Heap[i].freq,Heap[i].runLen)

def minHeapify(heap,i):
    l = len(heap) -1
    while l>i*2:
        if l < i*2+1:
            if heap[i].freq > heap[2*i].freq:
                heap[i], heap[2*i] = heap[2*i], heap[i]
                i = 2*i
        else:
            if heap[2*i].freq > heap[2*i+1].freq:
                child = 2*i+1
            else:
                child = 2*i
            if heap[i].freq > heap[child].freq:
                heap[i], heap[child] = heap[child], heap[i]
            i = child
    return heap

def insert(heap,Run):
    heap.append(Run)
    l = len(heap)-1
    i = l
    while i>1:
        parent = i//2
        if heap[parent].freq > heap[i].freq:
            heap[parent], heap[i] = heap[i], heap[parent]
            i = parent
        else:
            break
    return heap
    
#heap 만들고
Heap = makeHeap(runs)
while len(Heap) > 1:
    #젤작은거 두개 빼기
    min1, Heap = extractMin(Heap)
    min2, Heap = extractMin(Heap)

    #둘이 더해서 run 만들어서 추가
    newR = Run("",-1,min1.freq+min2.freq)
    newR.left = min1
    newR.right = min2

    Heap = insert(Heap,newR)
    minHeapify([0]+Heap,1)

printHeap([0]+Heap)
root = Heap[0]


##################################### 4. code word 부여하기  ###########################################
def assignCodewords(p,codeword,length):
    if p.left == None and p.right == None:
        p.codeword = int(codeword)
        p.codewordLen = length
    
    else:
        assignCodewords(p.left,codeword+"0",length+1)
        assignCodewords(p.right,codeword+"1",length+1)

def printTree(root):
    if root == None:
        return
    else:
        printTree(root.left)
        print(root.symbol, root.runLen,root.codeword,root.codewordLen)
        printTree(root.right)

assignCodewords(root,"",0)

##################################### 5. code word 검색하기  ###########################################
# dictionary 만들어서 character당 dictionary 만듦.
# 그 dic의 item은 리스트. 
codewordDic = {}
codeDic = {}
def storeCodeword(p):
    if p.left == None and p.right == None:
        if codewordDic.get(p.symbol) == None:
            codewordDic[p.symbol] = [[p.symbol,p.runLen,p.codeword,p.codewordLen,p.freq]]
            codeDic[p.codeword] = [p.symbol,p.runLen]
        else:
            codewordDic[p.symbol].append([p.symbol,p.runLen,p.codeword,p.codewordLen,p.freq])
            codewordDic[p.symbol].sort(key = lambda x:int(x[1]))
            codeDic[p.codeword] = [p.symbol,p.runLen]
    else:
        storeCodeword(p.left)
        storeCodeword(p.right)

storeCodeword(root)
print(codewordDic)

def findRun(dictt,symbol, length):
    for i in range(len(dictt[symbol])):
        if dictt[symbol][i][1] == length:
            return dictt[symbol][i][2]


print(findRun(codewordDic,"a",3))

##################################### 6. 인코딩 하기  ###########################################
# 원래 다 바이트 단위로 해야 진정한 압축이 됨.
# 하지만 너무 귀찮고 실제 상황도 아니므로 그냥 그대로 하겠음
# 나중에 바뀔점은 다 byte단위로 바꿔주기만 하면됨.

def outputFrequencies(fileName,runs):
    f = open(fileName,'r')
    text = f.read()
    f.close
    f = open(fileName,'w')
    f.write(str(len(runs)))
    f.write(str(os.path.getsize(fileName)))
    for i in range(len(runs)):
        f.write(runs[i].symbol)
        f.write(str(runs[i].runLen))
        f.write(str(runs[i].freq))
    f.close()
    return text
def encode(fileName,text,codewordDic):
    f = open(fileName,"a")
    
    start = 0
    i = 0
    ch = text[i]
    f.write("\n")
    # 한 byte를 읽어온다.
    while i < len(text): 
        ch = text[i]
        #지금이 시작이면
        if start == 0:
            startS = ch
            runL = 0
            start = 1
            #i+=1
            
        #계속이어짐    
        if ch == startS:
            runL += 1
            i+=1
        #run 끝남
        if ch != startS or i == len(text):
            start = 0
            code = findRun(codewordDic,startS,runL)
            f.write(str(code)+"\n")
    f.close()
text = outputFrequencies(fileName,runs)
print(len(text))
encode(fileName,text,codewordDic)

##################################### 7. 디코딩 하기  ###########################################
def decode(fileName):
    tmp = inputFrequencies(fileName)
    runs = []
    for i in range(len(tmp)):
        runs.append(Run(tmp[i][0],tmp[i][1],tmp[i][2]))

    runs.sort(key = lambda x: x.freq)
    Heap = makeHeap(runs)

    while len(Heap) > 1:
        #젤작은거 두개 빼기
        min1, Heap = extractMin(Heap)
        min2, Heap = extractMin(Heap)

        #둘이 더해서 run 만들어서 추가
        newR = Run("",-1,min1.freq+min2.freq)
        newR.left = min1
        newR.right = min2

        Heap = insert(Heap,newR)
        minHeapify([0]+Heap,1)

    printHeap([0]+Heap)
    root = Heap[0]
    print("트리:")
    printTree(root)
    assignCodewords(root,"",0)
    storeCodeword(root)

    f=open("newfile.txt",'w')
    rf = open(fileName,'r')
    rf.readline()
    line = rf.readline()
    
    while line:
        line = int(line[:-1])
        symbol = codeDic[line][0]
        runLen = codeDic[line][1]
        f.write(symbol*int(runLen))
        line = rf.readline()
    f.close()
    rf.close() 



def inputFrequencies(fileName):
    f=open(fileName,'r')
    line = f.readline()
    runNum = line[0]
    run = line[2:]
    runs = []
    for i in range(0,len(run),3):
        if run[i] == "\n"or run[i+1] == "\n":
            break
        symbol = run[i]
        runLen = run[i+1]
        freq = run[i+2]
        runs.append([symbol,runLen,freq])
    return runs


decode("txt.txt")


     

FileNotFoundError: ignored